In [24]:
import pandas as pd
import numpy as np

In [25]:
from pathlib import Path

In [26]:
dPath = Path("../docs/dumps")

In [27]:
import pickle

In [28]:
with open(dPath / "train_data.pkl", 'rb') as filename:
    train_data = pickle.load(filename)

In [29]:
with open(dPath / "valid_data.pkl", 'rb') as filename:
    valid_data = pickle.load(filename)

In [30]:
with open(dPath / "not_running.pkl", 'rb') as filename:
    not_running = pickle.load(filename)

In [31]:
X_train = train_data.drop("Detected", axis=1)

In [32]:
y_train = train_data.Detected

In [33]:
X_valid = valid_data.drop("Detected", axis=1)

In [34]:
y_valid = valid_data.Detected

In [35]:
with open(dPath / "rf_exp_03_names.pkl", 'rb') as filename:
    names = pickle.load(filename)

In [36]:
X_train = X_train[names] 

In [37]:
X_valid = X_valid[names]

In [38]:
X_train.head()

,numExecuted,MutatorClass,numAssertInTC,numTestCover,numAssertInTM,ppavcc,cchalsteadCumulativeBugs,ppRVF,ppnumberOfMethods,ppnumberOfClasses,...,returnType,ccfanIn,ppfanin,pploc,ccmaintainabilityIndexNC,mmexternalMethodsCalled,ppinstability,ppmaxcc,mmvariablesReferenced,ccunweightedClassSize
0,14,2,14,1,7,2.125,0.593518,13.0,16.0,5.0,...,3,1.0,1.0,262.0,99.687474,1.0,0.666667,5.0,3.0,4.0
1,14,6,14,1,7,2.125,0.593518,13.0,16.0,5.0,...,3,1.0,1.0,262.0,99.687474,1.0,0.666667,5.0,3.0,4.0
2,14,6,14,1,7,2.125,0.593518,13.0,16.0,5.0,...,3,1.0,1.0,262.0,99.687474,1.0,0.666667,5.0,3.0,4.0
3,14,0,14,1,7,2.125,0.593518,13.0,16.0,5.0,...,3,1.0,1.0,262.0,99.687474,1.0,0.666667,5.0,3.0,4.0
4,14,0,14,1,7,2.125,0.593518,13.0,16.0,5.0,...,3,1.0,1.0,262.0,99.687474,1.0,0.666667,5.0,3.0,4.0


In [39]:
from imblearn.over_sampling import ADASYN

In [40]:
sm = ADASYN(random_state=42, n_jobs=-1)

In [41]:
%time X_res, y_res = sm.fit_resample(X_train, y_train)

Wall time: 46.4 s


In [42]:
X_train_not_running = not_running.drop("Detected", axis=1)

In [43]:
y_train_not_running = not_running.Detected

In [44]:
X_train_not_running = X_train_not_running[names]

In [22]:
#X_train_not_running['ExecutedRatio'] = -1

In [23]:
#X_res['ExecutedRatio'] = np.log(X_res.numExecuted)/X_res.ppn

AttributeError: 'DataFrame' object has no attribute 'ppnumberOfStatements'

In [ ]:
X_train['ExecutedRatio'] = np.log(X_train.numExecuted)/X_train.ppnumberOfStatements

In [ ]:
X_valid['ExecutedRatio'] = np.log(X_valid.numExecuted)/X_valid.ppnumberOfStatements

In [45]:
X_res = pd.concat([X_res, X_valid], ignore_index=True,axis=0)

In [46]:
y_res = pd.concat([y_res, y_valid], ignore_index=True,axis=0)

In [47]:
y_res.sum()/y_res.shape[0]

0.5046457915994151

In [48]:
X_res.head()

,numExecuted,MutatorClass,numAssertInTC,numTestCover,numAssertInTM,ppavcc,cchalsteadCumulativeBugs,ppRVF,ppnumberOfMethods,ppnumberOfClasses,...,returnType,ccfanIn,ppfanin,pploc,ccmaintainabilityIndexNC,mmexternalMethodsCalled,ppinstability,ppmaxcc,mmvariablesReferenced,ccunweightedClassSize
0,14,2,14,1,7,2.125,0.593518,13.0,16.0,5.0,...,3,1.0,1.0,262.0,99.687474,1.0,0.666667,5.0,3.0,4.0
1,14,6,14,1,7,2.125,0.593518,13.0,16.0,5.0,...,3,1.0,1.0,262.0,99.687474,1.0,0.666667,5.0,3.0,4.0
2,14,6,14,1,7,2.125,0.593518,13.0,16.0,5.0,...,3,1.0,1.0,262.0,99.687474,1.0,0.666667,5.0,3.0,4.0
3,14,0,14,1,7,2.125,0.593518,13.0,16.0,5.0,...,3,1.0,1.0,262.0,99.687474,1.0,0.666667,5.0,3.0,4.0
4,14,0,14,1,7,2.125,0.593518,13.0,16.0,5.0,...,3,1.0,1.0,262.0,99.687474,1.0,0.666667,5.0,3.0,4.0


In [49]:
from sklearn.ensemble import RandomForestClassifier

In [50]:
rf = RandomForestClassifier(n_estimators=100, max_features=0.7, oob_score=True, random_state=42, n_jobs=-1)

In [51]:
%time rf.fit(X_res, y_res)

Wall time: 8min 44s


RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features=0.7,
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=-1, oob_score=True, random_state=42, verbose=0,
                       warm_start=False)

In [52]:
from sklearn.metrics import roc_auc_score
from sklearn.metrics import classification_report

In [53]:
def print_score(m):
    res = [m.score(X_train, y_train), m.score(X_valid, y_valid), roc_auc_score(y_valid, m.predict(X_valid))]
    if hasattr(m, 'oob_score_'): res.append(m.oob_score_)
    print(res)

In [54]:
def conf_matr(m):
    y_pred = m.predict(X_valid)
    print(classification_report(y_valid, y_pred))

In [55]:
print_score(rf)

[0.9080271791362952, 0.9207656750029631, 0.8800537794089773, 0.861780713116072]


In [56]:
conf_matr(rf)

              precision    recall  f1-score   support

       False       0.87      0.80      0.83     16657
        True       0.94      0.96      0.95     50839

    accuracy                           0.92     67496
   macro avg       0.90      0.88      0.89     67496
weighted avg       0.92      0.92      0.92     67496



In [77]:
with open(dPath / "rf_exp_09.pkl", 'wb') as filename:
    pickle.dump(rf,filename)

In [57]:
names = X_train.columns

In [58]:
feat_importances = pd.Series(rf.feature_importances_, index=names)
feat_importances.sort_values(ascending=False,inplace=True)

In [59]:
feat_importances

numExecuted                      0.147451
MutatorClass                     0.103115
mmhalsteadDifficulty             0.054694
numAssertInTM                    0.052156
numAssertInTC                    0.049819
numTestCover                     0.042716
cchalsteadCumulativeBugs         0.040792
mminstanceVariablesReferenced    0.040650
ppavcc                           0.040454
mmexternalMethodsCalled          0.039461
mmvariablesReferenced            0.038917
ccmaintainabilityIndex           0.025696
ccmaintainabilityIndexNC         0.025499
pploc                            0.024669
ccunweightedClassSize            0.022451
returnType                       0.022435
ppmaintainabilityIndexNC         0.021187
ccexternalMethodCalls            0.020899
ppmaintainabilityIndex           0.019931
ccimportedPackages               0.019593
ppmaxcc                          0.018010
ccfanIn                          0.017668
ppnumberOfClasses                0.017414
ppnumberOfMethods                0

In [43]:
with open(dPath / "rf_exp_08_names.pkl", 'wb') as filename:
    pickle.dump(names,filename)

In [61]:
with open(dPath / "test_data.pkl", 'rb') as filename:
    test_data = pickle.load(filename)

In [62]:
X_test = test_data.drop("Detected", axis=1)

In [46]:
X_test['ExecutedRatio'] = np.log(X_test.numExecuted)/X_test.ppnumberOfStatements

In [63]:
y_test = test_data.Detected

In [64]:
X_test = X_test[names]

In [65]:
with open(dPath / "mutclslabels.pkl", 'rb') as filename:
    mutclslabels = pickle.load(filename)

In [66]:
with open(dPath / "retypelabels.pkl", 'rb') as filename:
    retypelabels = pickle.load(filename)

In [67]:
mutclscodes = dict(zip(mutclslabels,range(len(mutclslabels))))

In [68]:
retypecodes = dict(zip(retypelabels,range(len(retypelabels))))

In [69]:
X_test.replace(mutclscodes, inplace=True)

In [70]:
X_test.replace(retypecodes, inplace=True)

In [71]:
preds = rf.predict_proba(X_test)[:,1]

In [72]:
np.histogram(preds)

(array([  603,   488,  1798,  4921, 11184, 14259, 11638,  7362,  3673,
         2550], dtype=int64),
 array([0. , 0.1, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8, 0.9, 1. ]))

In [73]:
print(classification_report(y_test, preds >=0.50))

              precision    recall  f1-score   support

       False       0.51      0.47      0.49     20633
        True       0.72      0.75      0.74     37843

    accuracy                           0.65     58476
   macro avg       0.62      0.61      0.61     58476
weighted avg       0.65      0.65      0.65     58476



In [74]:
from sklearn.metrics import average_precision_score, precision_score, recall_score
print(average_precision_score(y_test, preds >= 0.5, average='micro'))

0.7041866707664926


In [75]:
from sklearn import metrics
metrics.roc_auc_score(y_test, preds >= 0.5)

0.6118141757025684

In [76]:
with open(dPath / "rf_exp_09_preds.pkl", 'wb') as filename:
    pickle.dump(preds,filename)